In [2]:
# make a graph that shows convergence as a function of layout trials and swap trials

# very important point to not forget
# NOTE, these layout trials are after multiple SWAP trials and through forward-backwards pass
# this means it has already gone through 2 stages of optimization and we still have large variance

# ultimate question is how much time do we need to spend on swap restarts, forward-backward passes, and layout restarts
# maybe we work from bottom up, collect data to get a fair estimate of how much time we need to spend on each?

from qiskit.transpiler import CouplingMap
from mirror_gates.pass_managers import SabreMS, QiskitLevel3
from transpile_benchy.metrics.abc_metrics import MetricInterface
from transpile_benchy.metrics.gate_counts import DepthMetric
from mirror_gates.utilities import DoNothing

In [3]:
class LayoutTrialsStdMetric(MetricInterface):
    def __init__(self):
        """Initialize the metric."""
        super().__init__(
            name="layout_trials_std", pretty_name="Layout Trial Standard Deviation"
        )

    def _construct_pass(self):
        """Return the pass associated with this metric."""
        return DoNothing()


class LayoutTrialsMetric(MetricInterface):
    def __init__(self):
        """Initialize the metric."""
        super().__init__(name="layout_trials", pretty_name="Layout Trial Costs")

    def _construct_pass(self):
        """Return the pass associated with this metric."""
        return DoNothing()

In [5]:
from transpile_benchy.library import CircuitLibrary

# library = CircuitLibrary.from_txt("../medium_circuits.txt")
# library = CircuitLibrary.from_txt("iterations.txt")

In [12]:
coupling_map = CouplingMap.from_heavy_hex(5)
transpilers = [
    # QiskitLevel3(coupling_map),
    SabreMS(coupling_map, name="SABREMS", layout_trials=20)
]

metrics = [LayoutTrialsMetric(), DepthMetric(consolidate=False)]

In [13]:
coupling_map = CouplingMap.from_heavy_hex(5)
transpilers = [
    # QiskitLevel3(coupling_map),
    SabreMS(coupling_map, name="SABREMS", anneal_routing=False),
    SabreMS(coupling_map, name="SABREMS-Anneal", anneal_routing=True),
]

metrics = [LayoutTrialsMetric(), DepthMetric(consolidate=False)]

In [14]:
coupling_map = CouplingMap.from_heavy_hex(5)
transpilers = [
    # QiskitLevel3(coupling_map),
    SabreMS(coupling_map, name="SABREMS-MinSwaps", cost_function="basic"),
    SabreMS(coupling_map, name="SABREMS-MinDepth"),
]

metrics = [DepthMetric(consolidate=False)]  # , TotalMetric(consolidate=False)]

In [15]:
from transpile_benchy.benchmark import Benchmark

# only interested in TimeMetric, is there by default
benchmark = Benchmark(
    transpilers=transpilers,
    circuit_library=library,
    metrics=metrics,
    num_runs=10,
)
benchmark.run()

Circuits from library:   0%|          | 0/4 [00:00<?, ?it/s]

Loading qft_n8 from MQTBench


Circuits from library:  25%|██▌       | 1/4 [01:21<04:03, 81.05s/it]

Loading qft_n16 from MQTBench


Circuits from library:  50%|█████     | 2/4 [04:09<04:25, 132.60s/it]

Loading qft_n32 from MQTBench


Circuits from library: 100%|██████████| 4/4 [13:55<00:00, 208.92s/it]

Loading qft_n64 from MQTBench


In [18]:
print(benchmark)


Transpiler: SABREMS-MinDepth-$\sqrt{\texttt{iSWAP}}$

  Metric: monodromy_depth
  Circuit: qft_n16                                  Mean result: 63.543                         Trials: [70.5, 96.0, 73.0, 66.0, 42.5, 45.5, 71.0, 60.0, 42.5, 94.0]
  Circuit: qft_n32                                  Mean result: 188.362                        Trials: [171.5, 207.5, 187.5, 240.5, 127.0, 171.0, 182.5, 254.5, 235.5, 147.5]
  Circuit: qft_n8                                   Mean result: 21.089                         Trials: [20.5, 18.5, 22.5, 20.5, 18.5, 28.0, 18.5, 20.5, 22.5, 22.5]

  Metric: total_runtime
  Circuit: qft_n16                                  Mean result: 8.576                          Trials: [8.211485147476196, 8.583486318588257, 8.431179285049438, 8.396565675735474, 8.492676973342896, 8.351712226867676, 9.047089338302612, 8.211954832077026, 8.91487455368042, 9.121588945388794]
  Circuit: qft_n32                                  Mean result: 29.116                        

In [20]:
benchmark.metrics[0].saved_results

{'SABREMS-MinSwaps-$\\sqrt{\\texttt{iSWAP}}$': {'qft_n8': Result(monodromy_depth, 21.168977697667533),
  'qft_n16': Result(monodromy_depth, 68.92015195537596),
  'qft_n32': Result(monodromy_depth, 210.79832796050073)},
 'SABREMS-MinDepth-$\\sqrt{\\texttt{iSWAP}}$': {'qft_n8': Result(monodromy_depth, 21.08862360097365),
  'qft_n16': Result(monodromy_depth, 63.542659175402655),
  'qft_n32': Result(monodromy_depth, 188.361959494601)}}

In [17]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming benchmark.metrics[0].saved_results.items() is a dictionary
# where keys are circuit names and values are lists of costs for each trial
for k, v in benchmark.metrics[0].saved_results.items():
    print(k)
    for circuit, result in v.items():
        # print(circuit)
        # print(result.data)

        # find cumulative min trial-wise
        min_trials = np.minimum.accumulate(result.data, axis=1)
        print(min_trials.shape)
        # print(min_trials)

        # average element-wise across all trials
        avg = np.mean(min_trials, axis=0)
        # print(avg)

        # plot cumulative min vs trial
        # also plot average cumulative min vs trial
        # at each trial we also want to scatter plot each individual trial,

        # plot with latex
        with plt.style.context(["ipynb", "colorsblind10"]):
            plt.rcParams["text.usetex"] = True
            for j, trials in enumerate(min_trials):
                plt.plot(
                    trials,
                    color="blue",
                    alpha=0.1,
                    label="Individual Trials" if j == 0 else None,
                )
            plt.plot(avg, color="red", label="Average")
            plt.xlabel("Layout Trials")
            plt.ylabel("Cumulative Min Cost")
            plt.title(f"{k} {circuit}")
            plt.ticklabel_format(style="plain")  # Turn off scientific notation
            plt.legend()
            plt.show()

SABREMS-MinSwaps-$\sqrt{\texttt{iSWAP}}$


AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
# # explictly print results
# for k, v in benchmark.metrics[0].saved_results.items():
#     for circuit, result in v.items():
#         print(result.data)